# CIS 194: Homework 6

The *Fibonacci numbers* $F_n$ are defined as the sequence of integers,
beginning with 0 and 1, where every integer in the sequence is the
sum of the previous two. That is,

$$
F_0=0
$$
$$
F_1=1
$$
$$
F_n=F_{n-1} + F_{n-1} (n \ge 2)
$$

## Exercise 1

Translate the above definition of Fibonacci numbers directly into a
recursive function definition of type

```haskell
fib :: Integer -> Integer
```

so that `fib n` computes the *n*th Fibonacci number $F_n$.

Now use `fib` to define the *infinite list* of all Fibonacci numbers

```haskell
fibs1 :: [Integer] 
```

Try evaluating `fibs1` at the ghci prompt. You will probably get
bored watching it after the first 30 or so Fibonacci numbers, because
fib is ridiculously slow. Although it is a good way to *define* the Fibonacci numbers, it is not a very good way to *compute* them—in order
to compute $F_n$ it essentially ends up adding 1 to itself $F_n$ times! For
example, shown at right is the tree of recursive calls made by evaluating `fib 5`

As you can see, it does a lot of repeated work. In the end, fib
has running time $O(F_n)$, which (it turns out) is equivalent to $O(ϕ^n)$,
where $ ϕ =\frac{1+\sqrt{5}}{2} $
is the “golden ratio”. That’s right, the running time
is exponential in n. What’s more, all this work is also repeated from
each element of the list `fibs1` to the next. Surely we can do better

In [1]:
fib :: Integer -> Integer
fib 0 = 0
fib 1 = 1
fib n = fib (n - 1) + fib (n - 2)

In [2]:
fibs1 :: [Integer]
fibs1 = map fib [0..]

In [3]:
take 30 fibs1

[0,1,1,2,3,5,8,13,21,34,55,89,144,233,377,610,987,1597,2584,4181,6765,10946,17711,28657,46368,75025,121393,196418,317811,514229]

## Exercise 2

When I said “we” in the previous sentence I actually meant “you”.
Your task for this exercise is to come up with more efficient implementation. Specifically, define the infinite list

```haskell
fibs2 :: [Integer] 
```

so that it has the same elements as `fibs1`, but computing the first *n*
elements of `fibs2` requires only $O(n)$ addition operations. Be sure to
use standard recursion pattern(s) from the `Prelude` as appropriate.

In [4]:
fibs2 :: [Integer]
fibs2 = 0 : 1 : zipWith (+) fibs2 (drop 1 fibs2)

In [5]:
take 30 fibs2

[0,1,1,2,3,5,8,13,21,34,55,89,144,233,377,610,987,1597,2584,4181,6765,10946,17711,28657,46368,75025,121393,196418,317811,514229]

In [6]:
fibs2 !! 100000

...

## *Streams*
We can be more explicit about infinite lists by defining a type `Stream`
representing lists that *must be* infinite. (The usual list type represents
lists that *may* be infinite but may also have some finite length.)
In particular, streams are like lists but with *only* a “cons” constructor—
whereas the list type has two constructors, `[] (the empty list)` and
`(:) (cons)`, there is no such thing as an *empty stream*. So a stream is
simply defined as an element followed by a stream.

## Exercise 3

- Define a data type of polymorphic streams, `Stream`.
- Write a function to convert a `Stream` to an infinite list,
```haskell
streamToList :: Stream a -> [a] 
```
- To test your `Stream` functions in the succeeding exercises, it will be
useful to have an instance of `Show` for `Streams`. However, if you put
deriving `Show` after your definition of `Stream`, as one usually does,
the resulting instance will try to print an *entire* `Stream`—which,
of course, will never finish. Instead, you should make your own
instance of `Show` for `Stream`,
```haskell
instance Show a => Show (Stream a) where show ...
```
which works by showing only some prefix of a stream (say, the
first 20 elements).

In [7]:
data Stream a = Cons a (Stream a)

streamToList :: Stream a -> [a]
streamToList (Cons x xs) = x : streamToList xs

instance (Show a) => Show (Stream a) where
  show = (++ ", more...") . show . take 7 . streamToList

In [8]:
codeStream = code 0
  where
    code n = n `Cons` code (n * 8 + 6)

codeStream

[0,6,54,438,3510,28086,224694], more...

## Exercise 4

Let’s create some simple tools for working with Streams.

- Write a function
```haskell
streamRepeat :: a -> Stream a
```
which generates a stream containing infinitely many copies of the
given element.

In [9]:
streamRepeat :: a -> Stream a
streamRepeat x = x `Cons` streamRepeat x

In [10]:
streamRepeat 42

[42,42,42,42,42,42,42], more...

- Write a function
```haskell
streamMap :: (a -> b) -> Stream a -> Stream b
```
which applies a function to every element of a `Stream`.

In [11]:
streamMap :: (a -> b) -> Stream a -> Stream b
streamMap f (Cons x xs) = f x `Cons` streamMap f xs

In [12]:
streamMap (*2) codeStream

[0,12,108,876,7020,56172,449388], more...

- Write a function
```haskell
streamFromSeed :: (a -> a) -> a -> Stream a
```
which generates a Stream from a “seed” of type a, which is the
first element of the stream, and an “unfolding rule” of type `a -> a`
which specifies how to transform the seed into a new seed, to be
used for generating the rest of the stream.

In [13]:
streamFromSeed :: (a -> a) -> a -> Stream a
streamFromSeed f x = x `Cons` streamFromSeed f (f x)

In [14]:
streamFromSeed (*7) 1

[1,7,49,343,2401,16807,117649], more...

## Exercise 5

Now that we have some tools for working with streams, let’s create a few:

- Define the stream
```haskell
nats :: Stream Integer
```
which contains the infinite list of natural numbers 0, 1, 2, . . .

In [15]:
nats :: Stream Integer
nats = streamFromSeed (+ 1) 0

In [16]:
nats

[0,1,2,3,4,5,6], more...

- Define the stream
```haskell
ruler :: Stream Integer
```
which corresponds to the *ruler function*

$$
0, 1, 0, 2, 0, 1, 0, 3, 0, 1, 0, 2, 0, 1, 0, 4, . . .
$$

where the *n*th element in the stream (assuming the first element
corresponds to $n = 1$) is the largest power of 2 which evenly
divides *n*. 

In [17]:
interleave (Cons x xs) ys = Cons x (interleave ys xs)

ruler :: Stream Integer
ruler = interleave (streamRepeat 0) (streamMap (+ 1) ruler)

In [18]:
(take 17 . streamToList) ruler

[0,1,0,2,0,1,0,3,0,1,0,2,0,1,0,4,0]

## *Fibonacci numbers via generating functions (extra credit)*

This section is optional but *very cool*, so if you have time I hope you
will try it. We will use streams of Integers to compute the Fibonacci
numbers in an astounding way.

The essential idea is to work with *generating functions* of the form

$$
a_0 + a_1 x + a_2 x^2 + · · · + a_n x^n + . . .
$$

where $x$ is just a “formal parameter” (that is, we will never actually
substitute any values for $x$; we just use it as a placeholder) and all the
coefficients $a_i$ are integers. We will store the coefficients $a_0, a_1, a_2, . . .$
in a `Stream Integer`.

## Exercise 6 (Optional)

- First, define
```haskell
x :: Stream Integer
```
by noting that $x = 0 + 1x + 0x^2 + 0x^3 + . . . .$
- Define an instance of the `Num` type class for `Stream Integer`. Here’s what should go in your `Num` instance:
    - You should implement the `fromInteger` function. Note that $n = n + 0x + 0x^2 + 0x^3 + . . . .$
    - You should implement `negate`: to negate a generating function,negate all its coefficients.
    - You should implement (+), which works like you would expect:
$(a_0 + a_1 x + a_2 x^2 + . . .) + (b_0 + b_1 x + b_2 x^2 + . . .) = (a_0 + b_0) +(a_1 + b_1)x + (a_2 + b_2)x^2 + . . .$
    - Multiplication is a bit trickier. Suppose $A = a_0 + xA'$ and $B = b_0 + xB'$ are two generating functions we wish to multiply. We reason as follows:
$$
\begin{align}
AB &= (a_0 + xA')B \\
&= a_0B + xA'B \\
&= a_0(b_0 + xB') + xA'B \\
&= a_0 b_0 + x(a_0B' + A'B) \\
\end{align}
$$
    - That is, the first element of the product $AB$ is the product of
the first elements, $a_0 b_0$; the remainder of the coefficient stream
(the part after the $x$) is formed by multiplying every element in $B'$
(that is, the tail of $B$) by $a_0$, and to this adding the result of
multiplying $A'$ (the tail of $A$) by $B$.

Note that there are a few methods of the Num class I have not
told you to implement, such as abs and signum. ghc will complain
that you haven’t defined them, but don’t worry about it. We won’t
need those methods. (To turn off these warnings you can add

```haskell
{-# OPTIONS_GHC -fno-warn-missing-methods #-}
```

to the top of your file.)

If you have implemented the above correctly, you should be able
to evaluate things at the ghci prompt such as

```
*Main> x^4
*Main> (1 + x)^5
*Main> (x^2 + x + 3) * (x - 5)
```

- The penultimate step is to implement an instance of the `Fractional`
class for `Stream Integer`. Here the important method to define is
division, (/). I won’t bother deriving it (though it isn’t hard), but
it turns out that if $A = a_0 + xA'$ and $B = b_0 + xB'$
, then $A/B = Q$, where $Q$ is defined as
$$
Q = (a_0/b_0) + x((1/b_0)(A' − QB')
$$

That is, the first element of the result is $a_0/b_0$; the remainder is
formed by computing $A_0 − QB'$ and dividing each of its elements
by $b_0$.

Of course, in general, this operation might not result in a stream
of Integers. However, we will only be using this instance in cases
where it does, so just use the `div` operation where appropriate.

- Consider representing the Fibonacci numbers using a generating
function,
$$ 
F(x) = F_0 + F_1x + F_2x^2 + F_3x^3 + . . .
$$
Notice that $x + xF(x) + x^2F(x) = F(x)$:
$$
\begin{array}{r@{\quad}c@{\quad}c@{\quad}c@{\quad}c@{\quad}c@{\quad}c@{\quad}c}
    & x & & & & & & \\
    & F_0x & + & F_1x^2 & + & F_2x^3 & + & F_3x^4 + \dots \\
+   &      &   & F_0x^2 & + & F_1x^3 & + & F_2x^4 + \dots \\
\hline
0   & + x  & + & F_2x^2 & + & F_3x^3 & + & F_4x^4 + \dots
\end{array}
$$

Thus $x = F(x) − xF(x) − x^2F(x)$, and solving for $F(x)$ we find that
$$ 
F(x) = \frac{x}{1-x-x^2}
$$

Translate this into an (amazing, totally sweet) definition

```haskell
fibs3 :: Stream Integer
```

In [19]:
x :: Stream Integer
x = 0 `Cons` (1 `Cons` streamRepeat 0)

In [20]:
instance Num (Stream Integer) where
  (+) (Cons x xs) (Cons y ys) = Cons (x + y) (xs + ys)
  (*) (Cons x xs) b@(Cons y ys) = Cons (x * y) (streamMap (* x) ys + xs * b)
  fromInteger = (`Cons` streamRepeat 0)
  negate = streamMap negate

In [21]:
x^4

[0,0,0,0,1,0,0], more...

In [22]:
(1 + x)^5

[1,5,10,10,5,1,0], more...

In [23]:
(x^2 + x + 3) * (x - 5)

[-15,-2,-4,1,0,0,0], more...

In [24]:
instance Fractional (Stream Integer) where
  (/) a@(Cons x xs) b@(Cons y ys) = Cons (x `div` y) $ streamMap (`div` y) (xs - a / b * ys)

In [25]:
(4+12*x+9*x^2)/(2+3*x)

[2,3,0,0,0,0,0], more...

In [26]:
fibs3 :: Stream Integer
fibs3 = x / (1 - x - x ^ 2)

In [27]:
fibs3

[0,1,1,2,3,5,8], more...

In [28]:
(!!100) . streamToList $ fibs3

354224848179261915075

Notice fib y=1

In [29]:
qdiv :: Stream Integer -> Stream Integer -> Stream Integer
qdiv a@(Cons x xs) b@(Cons y ys) = Cons (x `div` y) (xs - a `qdiv` b * ys)

fibs3' :: Stream Integer
fibs3' = x `qdiv` (1 - x - x ^ 2)

In [30]:
fibs3'

[0,1,1,2,3,5,8], more...

In [31]:
(!!100) . streamToList $ fibs3'

354224848179261915075

## *Fibonacci numbers via matrices (extra credit)*

It turns out that it is possible to compute the nth Fibonacci number
with only $O(log n)$ (arbitrary-precision) arithmetic operations. This
section explains one way to do it.

Consider the 2 × 2 matrix $F$ defined by
$$
F=
\begin{bmatrix}
1 & 1 \\
1 & 0
\end{bmatrix}
$$

Notice what happens when we take successive powers of $F$ (see
http://en.wikipedia.org/wiki/Matrix_multiplication if you
forget how matrix multiplication works):

$$
F^2=
\begin{bmatrix}
2 & 1 \\
1 & 1
\end{bmatrix}
$$

$$
F^3=
\begin{bmatrix}
3 & 2 \\
2 & 1
\end{bmatrix}
$$

$$
F^4=
\begin{bmatrix}
5 & 3 \\
3 & 2
\end{bmatrix}
$$

$$
F^5=
\begin{bmatrix}
8 & 5 \\
5 & 3
\end{bmatrix}
$$

Curious! At this point we might well conjecture that Fibonacci numbers are involved, namely, that
$$
F^n=
\begin{bmatrix}
F_{n+1} & F_n \\
F_n & F_{n-1}
\end{bmatrix}
$$

for all $n ≥ 1$. Indeed, this is not hard to prove by induction on $n$.

The point is that exponentiation can be implemented in logarithmic time using a *binary exponentiation* algorithm. The idea is that to
compute $x^n$, instead of iteratively doing n multiplications of $x$, we compute
$$
f(x) = \begin{cases} 
      (x^{n/2})^2 & \text{n even} \\
      x(x^{(n-1)/2})^2 & \text{n odd}
\end{cases}
$$

where $x^{n/2}$ and $x^{(n−1)/2}$ are recursively computed by the same
method. Since we approximately divide $n$ in half at every iteration,
this method requires only $O(log n)$ multiplications.

The punchline is that Haskell’s exponentiation operator (^) *already
uses* this algorithm, so we don’t even have to code it ourselves!

## Exercise 7 (Optional)

- Create a type `Matrix` which represents $2 × 2$ matrices of `Integers`.
- Make an instance of the `Num` type class for `Matrix`. In fact, you only
have to implement the (*) method, since that is the only one we
will use. (If you want to play around with matrix operations a bit
more, you can implement `fromInteger`, `negate`, and `(+)` as well.)
- We now get fast (logarithmic time) matrix exponentiation for free,
since (^) is implemented using a binary exponentiation algorithm
in terms of (*). Write a function
```haskell
fib4 :: Integer -> Integer
```
which computes the nth Fibonacci number by raising $F$ to the nth
power and projecting out $F_n$ (you will also need a special case
for zero). Try computing the one millionth or even ten millionth
Fibonacci number.

In [32]:
type Matrix = (Integer, Integer, Integer, Integer)

instance Num Matrix where
  (*) (x1, x2, y1, y2) (m1, m2, n1, n2) = (x1 * m1 + y1 * m2, x1 * n1 + y1 * n2, x2 * m1 + y2 * m2, x2 * n1 + y2 * n2)

fib4 :: Integer -> Integer
fib4 n = let (_, _, _, res) = ((1, 1, 1, 0) :: Matrix) ^ n in res

In [33]:
take 17 $ map fib4 [1 ..]

[0,1,1,2,3,5,8,13,21,34,55,89,144,233,377,610,987]